In [ ]:
import torch

import data
import utils
import info_recorder as ir
import data_loader as dl
import initializer as init
import trainer as tn

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
df, size_info, size_dict = data.get_df()

In [ ]:
sourceCode_np = df.sourceCode.values
codeClass_np = df.classLabel.values

In [ ]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = utils.tokenize(sourceCode_np)
input_ids = utils.encode(tokenized_sourceCodes, ch2idx, max_len)

In [ ]:
ir.record_ch2idx(ch2idx)

In [ ]:
encoded_class2idx, class2idx, num_classes = utils.tokenize_encode_class(codeClass_np)

In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, encoded_class2idx, test_size = 0.2, random_state = 43
)

In [ ]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = dl.data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [ ]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('rnn/test1')

In [ ]:
# CNN-rand: Word vectors are randomly initialized.
tn.set_seed(42)
cnn_rand, optimizer = init.initilize_model(device=device,
                                        vocab_size=len(ch2idx),
                                        embed_dim=100,
                                        filter_sizes=[8, 8, 8, 8],
                                        num_filters=[100, 200, 200, 100],
                                        num_classes=len(class2idx),
                                        dropout=0.5,
                                        learning_rate=0.25,
                                        optimizerName="Adadelta",
                                        modelType="CNN")
                                        
tn.train(device, cnn_rand, optimizer, train_dataloader, '01_4Layer_8KZ', writer, val_dataloader, epochs=20)